In [ ]:
####################MODIFIABLE RUN PARAMETERS#########################    

wellslist=['A1','B1','C1']

tiprack_starting_pos ={
    "tiprack_10" : 'A1',
    "tiprack_300" : 'A1'
}

#Antibody incubation time in seconds
ABS_INCUBATION_TIME = 12*60*60

#debug mode skips all incubations, prints out additional info
debug = False

####################FIXED RUN PARAMETERS######################### 

API_VERSION= '2.7'
default_flow_rate = 50
well_flow_rate = 5
sample_flow_rate = 0.2
wash_volume = 150
USE_TROUGH = True

class Object:
    pass

####################LABWARE LAYOUT ON DECK#########################    
labwarePositions = Object()
labwarePositions.buffers_reservoir = 1
labwarePositions.par2 = 2
labwarePositions.antibodies_plate = 3
labwarePositions.tiprack_300 = 6


In [ ]:
####################GENERAL SETUP################################


from opentrons import execute, simulate
import string
import json
import time
import logging
logging.getLogger('opentrons').setLevel(logging.ERROR)


stats = Object()
stats.volume = 0



####################! FUNCTIONS - DO NOT MODIFY !######################### 
def washSamples(pipette, sourceSolutionWell, samples, volume, num_repeats=1):

    try:
        iter(samples)
        #print('samples are iterable')
    except TypeError:
        #print('samples arent iterable')
        samples = [samples]
    
    pipette.pick_up_tip()
    
    if(len(samples)==0):
        samples = [samples]
    print("Replacing solution on samples:" +str(samples) + " len=" + str(len(samples)))
    for i in range(0, num_repeats):
        print ("Iteration:"+ str(i))
        for s in samples:
            print ("Washing sample:" + str(s))
            pipette.aspirate(volume, sourceSolutionWell, rate=well_flow_rate)
            pipette.dispense(volume, s, rate=sample_flow_rate).blow_out()
            stats.volume += volume
    
    pipette.drop_tip()
    
def dilute_and_apply_fixative(pipette, sourceSolutionWell, dilutant_buffer_well, samples, volume):
    try:
        iter(samples)
        #print('samples are iterable')
    except TypeError:
        #print('samples arent iterable')
        samples = [samples]
    
    pipette.pick_up_tip()
    
    if(len(samples)==0):
        samples = [samples]
    print("Applying fixative to samples:" +str(samples) + " len=" + str(len(samples)))
 
    for s in samples:
        print ("Diluting fixative: " + str(s))
        pipette.aspirate(volume+50, dilutant_buffer_well, rate=well_flow_rate)
        pipette.dispense(volume+50, sourceSolutionWell, rate=well_flow_rate)
        for iterator in range(0, 3):
            print ("Mixing: " + str(iterator+1))
            pipette.aspirate(volume, sourceSolutionWell, rate=well_flow_rate)
            pipette.dispense(volume, sourceSolutionWell, rate=well_flow_rate)
        print ("Applying fixative to sample: " + str(s))
        pipette.aspirate(volume, sourceSolutionWell, rate=well_flow_rate)
        pipette.dispense(volume, s, rate=sample_flow_rate).blow_out()
        stats.volume += volume
    
    pipette.drop_tip()
    
def mix(pipette, sourceSolutionWell, volume, num_repeats):
   
    pipette.pick_up_tip()
    
    print("Mixing solution in samples:" +str(sourceSolutionWell))
    for i in range(0, num_repeats):
        print ("Iteration:"+ str(i))
        pipette.aspirate(volume, sourceSolutionWell, rate=2)
        pipette.dispense(volume, sourceSolutionWell, rate=2)
    
    pipette.drop_tip()


def countdown(t):
    while t:
        mins, secs = divmod(t, 60)
        timeformat = '{:02d}:{:02d}'.format(mins, secs)
        print(timeformat, end='\r')
        if (not debug): time.sleep(1)
        t -= 1
    print('off we go..')
    



In [ ]:
###########################LABWARE SETUP#################################

protocol = execute.get_protocol_api(API_VERSION) 

if debug: print(protocol)

tiprack_300 = protocol.load_labware('opentrons_96_tiprack_300ul', labwarePositions.tiprack_300, "tiprack 300ul")

if debug: print(tiprack_300)

pipette_300 =  protocol.load_instrument('p300_single', 'right', tip_racks = [tiprack_300])
pipette_300.flow_rate.dispense = default_flow_rate
pipette_300.flow_rate.aspirate = default_flow_rate
pipette_300.starting_tip = tiprack_300.well(tiprack_starting_pos['tiprack_300'])

if debug: print(pipette_300)
    

par2_slides = protocol.load_labware('par2_slides', labwarePositions.par2, 'par2_slides')

#trough12_def = json.loads(AXYGEN_12well_plate_DEF_JSON)

#trough12 = protocol.load_labware_from_definition(trough12_def, labwarePositions.buffers_reservoir, '12-trough buffers reservoir')

#custom_96_def = json.loads(CUSTOM_96well_plate_DEF_JSON)


custom_96 = protocol.load_labware('black_96', labwarePositions.antibodies_plate, '96-well-plate')

trough12 = protocol.load_labware('axygen_12well_trough', labwarePositions.buffers_reservoir, 'Axygen_12well_trough')

temp_mod = protocol.load_module('temperature module', '8')

par2_on_heat_module=temp_mod.load_labware('par2_slides')


if debug: print(par2)
    


In [ ]:
buffer_wells = trough12.wells_by_name()

buffers = Object()
buffers.retreaval =  buffer_wells['A1']
buffers.TBS_wash =  buffer_wells['A2']
buffers.water =  buffer_wells['A3']
buffers.storage = buffer_wells['A4']


preblock_wells = custom_96.rows()[0]
antibody_wells = custom_96.rows()[1]
enzymeblock_wells = custom_96.rows()[2]
hrpsecondaryab_wells = custom_96.rows()[3]
substrate_wells = custom_96.rows()[4]
DAB_wells = custom_96.rows()[5]

sample_chambers = []

for well in wellslist:
    sample_chambers.append(par2_on_heat_module.wells_by_name()[well])

if debug: print(sample_chambers)


In [ ]:
#################PROTOCOL####################
protocol.home()

In [ ]:
#pipette_300.drop_tip()

In [ ]:
temp_mod.set_temperature(95)

In [ ]:
print("retreaval")
countdown(2400)

In [ ]:
temp_mod.set_temperature(25)

In [ ]:
#temp_mod.deactivate()

In [ ]:
print("retreaval")
countdown(1200)

In [ ]:
#WASHING SAMPLES WITH TBS
print("washing in TBS")
washSamples(pipette_300, buffers.TBS_wash, sample_chambers, wash_volume, num_repeats=2)
#countdown(300)

In [ ]:
#pipette_300.drop_tip()

In [ ]:
print("preblocking")
for i in range (len(wellslist)):
    washSamples(pipette_300, preblock_wells[i], sample_chambers[i], wash_volume)
#INCUBATE 15 MIN
print("preblocking incubation: 15 min")
countdown(3600)


In [ ]:
#countdown(3600)

In [ ]:
#APPLYING ANTIBODY COCKTAILS TO SAMPLES
print("applying antibodies")
for i in range (len(wellslist)):
    washSamples(pipette_300, antibody_wells[i], sample_chambers[i], wash_volume)

#INCUBATE 120 MIN
print("staining incubation: " + str(ABS_INCUBATION_TIME/3600) +"h" )
countdown(ABS_INCUBATION_TIME)

In [ ]:
#washSamples(pipette_300, buffers.TBS_wash, sample_chambers, wash_volume, num_repeats=1)

In [ ]:
#WASHING SAMPLES WITH TBS
#three individual repeats below is because they need particular incubation time between them
print("washing with TBS")
washSamples(pipette_300, buffers.TBS_wash, sample_chambers, wash_volume, num_repeats=1)
countdown(180)
washSamples(pipette_300, buffers.TBS_wash, sample_chambers, wash_volume, num_repeats=1)
countdown(180)
washSamples(pipette_300, buffers.TBS_wash, sample_chambers, wash_volume, num_repeats=1)
countdown(180)
washSamples(pipette_300, buffers.TBS_wash, sample_chambers, wash_volume, num_repeats=1)
countdown(180)
washSamples(pipette_300, buffers.TBS_wash, sample_chambers, wash_volume, num_repeats=1)
countdown(180)

In [ ]:
#APPLYING enzyme blocking
print("applying enzyme blocking")
for i in range (len(wellslist)):
    washSamples(pipette_300, enzymeblock_wells[i], sample_chambers[i], wash_volume)

#INCUBATE 120 MIN
print("hrp blocking incubation: 10min" )
countdown(600)

In [ ]:
washSamples(pipette_300, buffers.TBS_wash, sample_chambers, wash_volume, num_repeats=2)
countdown(180)
washSamples(pipette_300, buffers.TBS_wash, sample_chambers, wash_volume, num_repeats=2)
countdown(180)

In [ ]:
#APPLYING HRP SECONDARY ANTIBODY COCKTAILS TO SAMPLES
print("applying hrpsecondaryab")
for i in range (len(wellslist)):
    washSamples(pipette_300, hrpsecondaryab_wells[i], sample_chambers[i], wash_volume)

#INCUBATE 120 MIN
print("staining incubation: " + str(ABS_INCUBATION_TIME/3600) +"h" )
countdown(1800)

In [ ]:
#three individual repeats below is because they need particular incubation time between them
print("washing with TBS")
washSamples(pipette_300, buffers.TBS_wash, sample_chambers, wash_volume, num_repeats=1)
countdown(180)
washSamples(pipette_300, buffers.TBS_wash, sample_chambers, wash_volume, num_repeats=1)
countdown(180)
washSamples(pipette_300, buffers.TBS_wash, sample_chambers, wash_volume, num_repeats=1)
countdown(180)

In [ ]:
#DILUTING AND APPLYING THE DAB
for i in range (len(wellslist)):
    dilute_and_apply_fixative(pipette_300, DAB_wells[i], substrate_wells[i], sample_chambers[i], 200)
    
print("developing substrate")

countdown(600)

In [ ]:
washSamples(pipette_300, buffers.water, sample_chambers, wash_volume, num_repeats=5)
countdown(180)

In [ ]:
#STORAGE, washing samples every hour
for i in range (48):
    washSamples(pipette_300, buffers.storage,sample_chambers, 100)
    countdown(3600)
    print("total dispensed volume: ", str(stats.volume))